<a href="https://colab.research.google.com/github/mflzkejxmglkzjefmlkgjzmlefxjkg/test/blob/master/workertesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%javascript

console.log('Test SW');

// console.log(document.location);

// =================== BEGIN TEST 1 =================== (not working)
// https://stackoverflow.com/questions/21012580/is-it-possible-to-write-data-to-file-using-only-javascript
// var textFile = null,
// makeTextFile = function (text) {
//   var data = new Blob([text], {type: 'application/javascript'});
//   if (textFile !== null) {
//     window.URL.revokeObjectURL(textFile);
//   }
//   textFile = window.URL.createObjectURL(data);
//   return textFile;
// };

var script = `
// function logMessage(event){
//   try {
//     console.log("origin: " + event.origin + " source: ", event.source, "data:", event.data);
//   } catch (error) {
//     console.log("origin: " + event.origin + " source is cross-origin", "data:", event.data);
//   }
// }
// addEventListener("message", logMessage);
// ======
// (function() {
//     this.setTimeout(function() {
//         this.parent.postMessage(this.JSON.stringify({"selfLink": window.colabConfig}), "*");
//     }, 10);
// })();
// ======
// console.log('BEGIN');
// importScripts('https://apis.google.com/js/googleapis.proxy.js?onload=startup');
`;

// // console.log(makeTextFile(script));
// // https://stackoverflow.com/questions/39136625/service-worker-registration-failed
// if('serviceWorker' in navigator){
//   navigator.serviceWorker.register(makeTextFile(script).replace('blob:','')).then(function(reg){
//     console.log("SW registration succeeded. Scope is "+reg.scope);
//   }).catch(function(err){
//     console.error("SW registration failed with error "+err);
//   });
// }
// =================== END TEST 1 ===================

// =================== BEGIN TEST 2 =================== (partially working)
// https://medium.com/@onejohi/service-workers-in-depth-f5c9993ab8e7
// let workerstring = '(function(){console.log(window);})();'

let workerstring = script;
let blobURL = URL.createObjectURL(new Blob([workerstring], { type: 'text/javascript' }));
let webworker = new Worker(blobURL);

// =================== END TEST 2 ===================

// =================== BEGIN TEST 3 =================== (not working)
// if ('serviceWorker' in navigator) {
//   // Register a service worker hosted at the root of the
//   // site using the default scope.
//   navigator.serviceWorker.register(blobURL).then(function(registration) {
//     console.log('Service worker registration succeeded:', registration);
//   }, /*catch*/ function(error) {
//     console.log('Service worker registration failed:', error);
//   });
// } else {
//   console.log('Service workers are not supported.');
// }
// =================== END TEST 3 ===================

<IPython.core.display.Javascript object>

In [ ]:
%%js

console.log(google.colab);

<IPython.core.display.Javascript object>

In [3]:
import http.server
import socketserver
import portpicker
import threading
import os
from google.colab import output
from IPython.display import Javascript

started = threading.Event()
port = portpicker.pick_unused_port()

# Start a background thread serving up the files.
def server_entry():
  os.chdir('/content/test/')
  with socketserver.TCPServer(("", port), http.server.SimpleHTTPRequestHandler) as httpd:
    started.set()
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()
started.wait();

# Tell the JS side the port #
output.eval_js(f"window.mnistPort = {port}")

Javascript('''
  (async () => {
    if (!google.colab.kernel.accessAllowed) {
        // User needs to execute the notebook.
        return;
    }
    const url = await google.colab.kernel.proxyPort(window.mnistPort);
    const iframe = document.createElement('iframe');
    iframe.src = url;
    iframe.height = '500px';
    iframe.width = '100%';
    iframe.style.border = '0';
    document.body.appendChild(iframe);
  })()
''')

<IPython.core.display.Javascript object>